In [20]:
import Game
import ScoreBoard
import BoxScore

import importlib
importlib.reload(Game)
importlib.reload(ScoreBoard)
importlib.reload(BoxScore)

from ipywidgets import widgets, HTML
output = widgets.Output()
display(output)

import pandas as pd
import glob
import os

allfiles = glob.glob('./data/*.game')
phases = sorted(list(set([os.path.basename(x).split('-')[1] for x in allfiles if '-' in x and '.a-' in x])))

allevents = []
sb = ScoreBoard.ScoreBoard('./data/Urbania.team', scale=0.4, output=output)

# Conversion to int without errors
def toint(x):
    try:
        return int(x)
    except:
        return 999999

progressive = 1
for phase in phases:

    # File of the phase
    files = [x for x in allfiles if phase in x and os.path.basename(x).split('-')[1] == phase]

    # Sort on increasing phase + round
    rounds = [toint(os.path.basename(x).split('.')[0]) for x in files]
    files  = [file for r, file in sorted(zip(rounds, files))]

    for file in files:
        sb.game.loadGame(file)
        df = sb.game.events_df.copy()
        if df.shape[0] > 0:
            name = file.replace('./data/','').replace('.game','')
            elems = name.split('-')
            ro = int(elems[0].replace('.a',''))
            ph = elems[1]
            op = elems[2]
            df['game_number'] = progressive
            df['round'] = ro
            df['phase'] = ph
            df['opponents'] = op
            allevents.append(df)
            
        progressive += 1

df = pd.concat(allevents)
df.reset_index(drop=True, inplace=True)
df

Output()

,team,player,event,event_name,event_description,quarter,seconds,x,y,time,game_number,round,phase,opponents
0,Team,Di Francesco,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO
1,Team,Colotti,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO
2,Team,Dusels,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO
3,Team,Aloi,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO
4,Team,Diana,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5372,Team,Diana,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO
5373,Team,Aloi,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO
5374,Team,Colotti,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO
5375,Team,Pentucci,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO


In [28]:
svg = BoxScore.svg(df, game=sb.game, width=55.0)
HTML(svg)

HTML(value='<svg version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" x…